# This is the first notebook of the project. It trains a bare-minimum network for face aging.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp '/content/drive/My Drive/UTK_inthewild/part1.tar.gz' .
!cp '/content/drive/My Drive/UTK_inthewild/part2.tar.gz' .
!cp '/content/drive/My Drive/UTK_inthewild/part3.tar.gz' .

In [0]:
!mkdir images
!tar xzf part1.tar.gz -C images
!tar xzf part2.tar.gz -C images
!tar xzf part3.tar.gz -C images
!mv images/part1/* images
!mv images/part2/* images
!mv images/part3/* images

In [0]:
!rm -r images/part1
!rm -r images/part2
!rm -r images/part3

In [0]:
!ls -l images | wc -l

24109


In [0]:
import tensorflow as tf

tf.enable_eager_execution()

import tensorflow.keras as keras
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

import tensorflow.keras.backend as K

import numpy as np
import cv2
import os
import gc

In [0]:
class InstanceNormalization(Layer):
    def __init__(self, **kwargs):
        super(InstanceNormalization, self).__init__()

    def build(self, input_shape):
        depth = input_shape[3]
        self.scale = self.add_weight(name="scale_"+str(depth),
                                     shape=[depth], 
                                     initializer=tf.random_normal_initializer(1.0, 0.02, dtype=tf.float32),
                                     dtype=tf.float32
                                    )
            
        self.offset = self.add_weight(name="offset_"+str(depth),
                                      shape=[depth], 
                                      initializer=tf.constant_initializer(0.0),
                                      dtype=tf.float32
                                    )
        super(InstanceNormalization, self).build(input_shape)
        
    def call(self, input_):
        mean, variance = tf.nn.moments(input_, axes=[1,2], keep_dims=True)
        epsilon = 1e-5
        inv = tf.rsqrt(variance + epsilon)
        normalized = (input_-mean)*inv
        return self.scale*normalized + self.offset

In [0]:
class Residual(Layer):
    def __init__(self, dim=256, ks=3, s=1, padding='VALID', stddev=0.02, from_list=None, **kwargs):
        self.dim = dim
        self.ks = ks
        self.s = s
        self.from_list = from_list
        self.padding=padding
        self.stddev=stddev
        super(Residual, self).__init__()

    def build(self, input_shape):
        self.conv2d1 = Conv2D(filters=self.dim, kernel_size=self.ks, strides=self.s, padding=self.padding, activation=None,
                            kernel_initializer=tf.truncated_normal_initializer(stddev=self.stddev),
                            bias_initializer=None)
        self.instnorm1 = InstanceNormalization()
        self.relu1 = ReLU()
        self.conv2d2 = Conv2D(filters=self.dim, kernel_size=self.ks, strides=self.s, padding=self.padding, activation=None,
                            kernel_initializer=tf.truncated_normal_initializer(stddev=self.stddev),
                            bias_initializer=None)
        self.instnorm2 = InstanceNormalization()
        
        super(Residual, self).build(input_shape)
        
    def call(self, x):
        p = int((self.ks - 1) / 2)
        y = tf.pad(x, [[0, 0], [p, p], [p, p], [0, 0]], "REFLECT")

        y = self.instnorm1(self.conv2d1(y))
        y = tf.pad(self.relu1(y), [[0, 0], [p, p], [p, p], [0, 0]], "REFLECT")
        y = self.instnorm2(self.conv2d2(y))
        return y + x 


In [0]:
def instance_norm(inp, name="instance_norm"):
    return InstanceNormalization()(inp)

In [0]:
def conv2d(input_, output_dim, ks=4, s=2, stddev=0.02, padding='SAME', name="conv2d"):
    return Conv2D(filters=output_dim, kernel_size=ks, strides=s, padding=padding, activation=None,
                            kernel_initializer=tf.truncated_normal_initializer(stddev=stddev),
                            bias_initializer=None)(input_)

def deconv2d(input_, output_dim, ks=4, s=2, stddev=0.02, name="deconv2d"):
    return Conv2DTranspose(filters=output_dim, kernel_size=ks, strides=s, padding='SAME', activation=None,
                                    kernel_initializer=tf.truncated_normal_initializer(stddev=stddev),
                                    bias_initializer=None)(input_)

def lrelu(x, leak=0.2, name="lrelu"):
    return LeakyReLU(leak)(x)

@tf.function
def abs_criterion(in_, target):
    return tf.reduce_mean(tf.abs(in_ - target))

@tf.function
def mae_criterion(in_, target):
    return tf.reduce_mean((in_-target)**2)

@tf.function
def sce_criterion(logits, labels):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))


In [0]:
for d in os.listdir('.'):
    if d[0].isdigit():
        os.system('rm -r ' + d)

In [0]:
def make_discriminator(name="discriminator"):

    inp = Input((128, 128, 3))
    df_dim = 64

    h0 = lrelu(conv2d(inp, df_dim, name='d_h0_conv'))
    # h0 is (128 x 128 x self.df_dim)
    h1 = lrelu(instance_norm(conv2d(h0, df_dim*2, name='d_h1_conv'), 'd_bn1'))
    # h1 is (64 x 64 x self.df_dim*2)
    h2 = lrelu(instance_norm(conv2d(h1, df_dim*4, name='d_h2_conv'), 'd_bn2'))
    # h2 is (32x 32 x self.df_dim*4)
    h3 = lrelu(instance_norm(conv2d(h2, df_dim*8, s=1, name='d_h3_conv'), 'd_bn3'))
    # h3 is (32 x 32 x self.df_dim*8)
    h4 = conv2d(h3, 1, s=1, name='d_h3_pred')
    # h4 is (32 x 32 x 1)
    m = Model(inputs=[inp], outputs=[h4])
    return m

def make_generator_resnet(name="generator"):

    inp = Input((128, 128, 3))
    gf_dim = 64
    output_c_dim = 3

    c0 = tf.pad(inp, [[0, 0], [3, 3], [3, 3], [0, 0]], "REFLECT")
    c1 = ReLU()(instance_norm(conv2d(c0, gf_dim, 7, 1, padding='VALID', name='g_e1_c'), 'g_e1_bn'))
    c2 = ReLU()(instance_norm(conv2d(c1, gf_dim*2, 3, 2, name='g_e2_c'), 'g_e2_bn'))
    c3 = ReLU()(instance_norm(conv2d(c2, gf_dim*4, 3, 2, name='g_e3_c'), 'g_e3_bn'))        # 64 *64
    
    r1 = Residual(dim=gf_dim*4, name='g_r1')(c3)
    r2 = Residual(dim=gf_dim*4, name='g_r2')(r1)
    r3 = Residual(dim=gf_dim*4, name='g_r3')(r2)
    r4 = Residual(dim=gf_dim*4, name='g_r4')(r3)
    r5 = Residual(dim=gf_dim*4, name='g_r5')(r4)
    r6 = Residual(dim=gf_dim*4, name='g_r6')(r5)
    r7 = Residual(dim=gf_dim*4, name='g_r7')(r6)
    r8 = Residual(dim=gf_dim*4, name='g_r8')(r7)
    r9 = Residual(dim=gf_dim*4, name='g_r9')(r8)

    d1 = deconv2d(r9, gf_dim*2, 3, 2, name='g_d1_dc')
    d1 = ReLU()(instance_norm(d1, 'g_d1_bn'))
    d2 = deconv2d(d1, gf_dim, 3, 2, name='g_d2_dc')
    d2 = ReLU()(instance_norm(d2, 'g_d2_bn'))
    d2 = tf.pad(d2, [[0, 0], [3, 3], [3, 3], [0, 0]], "REFLECT")

    pred = Activation("tanh")(conv2d(d2, output_c_dim, 7, 1, padding='VALID', name='g_pred_c'))
    m = Model(inputs=[inp], outputs=[pred])
    return m

In [0]:
model_G = make_generator_resnet("G")                   # First Generator
model_H = make_generator_resnet("H")                   # Second Generator
model_D = make_discriminator("D")                   # Discriminator for G
model_E = make_discriminator("E")                   # Discriminator for H

if os.path.exists('/content/drive/My Drive/UTK_inthewild/model_D_age.h5'):
    print("Loading model_D")
    model_D.load_weights('/content/drive/My Drive/UTK_inthewild/model_D_age.h5')
    
if os.path.exists('/content/drive/My Drive/UTK_inthewild/model_E_age.h5'):
    print("Loading model_E")
    model_E.load_weights('/content/drive/My Drive/UTK_inthewild/model_E_age.h5')
    
if os.path.exists('/content/drive/My Drive/UTK_inthewild/model_G_age.h5'):
    print("Loading model_G")
    model_G.load_weights('/content/drive/My Drive/UTK_inthewild/model_G_age.h5')
    
if os.path.exists('/content/drive/My Drive/UTK_inthewild/model_H_age.h5'):
    print("Loading model_H")
    model_H.load_weights('/content/drive/My Drive/UTK_inthewild/model_H_age.h5')

W0812 10:09:41.283514 139718129379200 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/converters/directives.py:117: The name tf.rsqrt is deprecated. Please use tf.math.rsqrt instead.



Loading model_D
Loading model_E
Loading model_G
Loading model_H


In [0]:
model_H.summary()
model_E.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
tf_op_layer_MirrorPad_2 (Ten [(None, 134, 134, 3)]     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 128, 64)      9472      
_________________________________________________________________
instance_normalization_5 (In (None, 128, 128, 64)      128       
_________________________________________________________________
re_lu_5 (ReLU)               (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 64, 128)       73856     
_________________________________________________________________
instance_normalization_6 (In (None, 64, 64, 128)       256 

In [0]:
def get_image_list(path='images', young_lower=18, young_upper=30, old_lower=50, old_upper=80):
    images = os.listdir(path)
    ages = {
        'young': [],
        'old': []
    }
    
    for image in images:
        age = int(image.split('_')[0])
        if young_lower <= age <= young_upper:
            ages['young'].append(image)
        if old_upper >= age >= old_lower:
            ages['old'].append(image)
        
    return ages

image_dataset = get_image_list()

In [0]:
from sklearn.utils import shuffle

def generator(gen_func, *args):
    while True:
        yield gen_func(*args)
        
def create_young_old_batch(batch_size=32, shuffle_data=True):
    X1 = []
    X2 = []
    x = 128
    images = get_image_list()['young']
    to_create = np.random.choice(images, batch_size)
    for img in to_create:
        image = cv2.imread('images/' + img)
        image = cv2.resize(image, (x, x))
        X1.append(image)
        
    images = get_image_list()['old']
    to_create = np.random.choice(images, batch_size)
    for img in to_create:
        image = cv2.imread('images/' + img)
        image = cv2.resize(image, (x, x))
        X2.append(image)
        
    X1 = np.stack(X1).astype(np.float32)
    X2 = np.stack(X2).astype(np.float32)
    
    if shuffle_data:
        X1, X2 = shuffle(X1, X2)
        
    return (X1.astype(np.float32) / 127.5) - 1.0, (X2.astype(np.float32) / 127.5) - 1.0

def create_young_old_test_batch(batch_size=32, shuffle_data=True):
    X1 = []
    X2 = []
    size = 128
    images = get_image_list()['young']
    to_create = images[:batch_size]
    for img in to_create:
        image = cv2.imread('images/' + img)
        image = cv2.resize(image, (size, size))
        X1.append(image)
        
    images = get_image_list()['old']
    to_create = images[:batch_size]
    for img in to_create:
        image = cv2.imread('images/' + img)
        image = cv2.resize(image, (size, size))
        X2.append(image)
        
    X1 = np.stack(X1).astype(np.float32)
    X2 = np.stack(X2).astype(np.float32)
    
    if shuffle_data:
        X1, X2 = shuffle(X1, X2)
        
    return (X1.astype(np.float32) / 127.5) - 1., (X2.astype(np.float32) / 127.5) - 1.

# Below are the cells used for training. `GradientTape` is used instead of `model.fit()`

In [0]:
g1_optimizer = tf.train.AdadeltaOptimizer(1.0)
g2_optimizer = tf.train.AdadeltaOptimizer(1.0)
d1_optimizer = tf.train.AdadeltaOptimizer(1.0)
d2_optimizer = tf.train.AdadeltaOptimizer(1.0)

# disc_loss_fn = tf.keras.losses.MeanSquaredError()
# gen_loss_fn = tf.keras.losses.MeanSquaredError()
# cycled_loss_fn = tf.keras.losses.MeanAbsoluteError()
identity_loss_fn = tf.keras.losses.MeanAbsoluteError()

def discriminator_loss(real, generated):
    real_loss = disc_loss_fn(tf.ones_like(real), real)
    generated_loss = disc_loss_fn(tf.zeros_like(generated), generated)
    total_disc_loss = real_loss + generated_loss
    return total_disc_loss

def cycled_loss(real, cycled):
    return cycled_loss_fn(real, cycled)

def identity_loss(real, same):
    return identity_loss_fn(real, same)

def generator_loss(generated):
  return gen_loss_fn(tf.ones_like(generated), generated)

@tf.function
def train(genAB, genBA, discA, discB, X1, X2, lambda1=10.0):
    real_A = X1
    real_B = X2
    with tf.GradientTape(persistent=True) as tape:
        fake_B = genAB(real_A)                  # Young -> old
        fake_A = genBA(real_B)                  # Old -> young
        
        cycled_A = genBA(fake_B)            # Young -> old -> young
        cycled_B = genAB(fake_A)            # old -> young -> old
        
        same_B = genAB(real_B)                 # Old -> Old
        same_A = genBA(real_A)                 # Young -> Young
        
        discA_fake_out = discA(fake_A)        # D predicts young, gen2, ie H, outputs young
        discB_fake_out = discB(fake_B)

        g_loss = mae_criterion(discA_fake_out, tf.ones_like(discA_fake_out)) \
                + mae_criterion(discB_fake_out, tf.ones_like(discB_fake_out)) \
                + lambda1 * abs_criterion(real_A, cycled_A) \
                + lambda1 * abs_criterion(real_B, cycled_B)  \
                + identity_loss(cycled_A, same_A) \
                + identity_loss(cycled_B, same_B)
        
        # cycle_loss = cycled_loss(X1, cycled_x) + cycled_loss(X2, cycled_y)
        
        discA_real_out = discA(real_A)
        discB_real_out = discB(real_B)

        discA_loss_real = mae_criterion(discA_real_out, tf.ones_like(discA_real_out))
        discB_loss_real = mae_criterion(discB_real_out, tf.ones_like(discB_real_out))

        discA_loss_fake = mae_criterion(discA_fake_out, tf.zeros_like(discA_fake_out))
        discB_loss_fake = mae_criterion(discB_fake_out, tf.zeros_like(discB_fake_out))

        discA_loss = (discA_loss_real + discA_loss_fake) / 2.0
        discB_loss = (discB_loss_real + discB_loss_fake) / 2.0

        d_loss = discA_loss + discB_loss
                
        net_loss = g_loss + d_loss
           
    G_vars = genAB.trainable_variables
    gradients = tape.gradient(g_loss, G_vars)
    g1_optimizer.apply_gradients(zip(gradients, G_vars))
    
    H_vars = genBA.trainable_variables
    gradients = tape.gradient(g_loss, H_vars)
    g2_optimizer.apply_gradients(zip(gradients, H_vars))
    
    D_vars = discA.trainable_variables
    gradients = tape.gradient(d_loss, D_vars)
    d1_optimizer.apply_gradients(zip(gradients, D_vars))
    
    E_vars = discB.trainable_variables
    gradients = tape.gradient(d_loss, E_vars)
    d2_optimizer.apply_gradients(zip(gradients, E_vars))
    
    del tape
    gc.collect()
    return net_loss, g_loss, d_loss

In [0]:
from tensorflow.keras.utils import Progbar

In [0]:
gc.collect()
for epoch in range(10):
    
    print("Epoch ", epoch)
    batch_size=8
    steps = int((len(image_dataset['young']) + len(image_dataset['old']))/batch_size)
    progressbar = Progbar(steps)
    for i in range(steps) :
        # X1, X2 = create_young_old_batch(batch_size=32)
        # loss = train_discriminators(model_G, model_H, model_D, model_E, X1, X2)
        # gc.collect()
        X1, X2 = create_young_old_batch(batch_size=batch_size)
        loss, gl, dl = train(model_G, model_H, model_D, model_E, X1, X2)
        progressbar.update(i, [
            ('loss', loss.numpy()),
            ('generator loss', gl.numpy()),
            ('discriminator loss', dl.numpy())
        ])
        
        if i%100 == 0:
            os.mkdir(str(epoch) + '_' + str(i))
            os.mkdir(str(epoch) + '_' + str(i) + '/young2old')
            os.mkdir(str(epoch) + '_' + str(i) + '/old2young')
        
            Xy, Xo = create_young_old_test_batch(batch_size=8)
            Yy = np.around((1 + model_G.predict(Xy)) * 127.5)
            Yo = np.around((1 + model_H.predict(Xo)) * 127.5)
            Xy = np.around((1 + Xy) * 127.5)
            Xo = np.around((1 + Xo) * 127.5)
        
            for j in range(len(Yy)):
                cv2.imwrite(str(epoch) + '_' + str(i) + '/young2old/' + str(j) + '.jpg', Yy[j])
                cv2.imwrite(str(epoch) + '_' + str(i) + '/old2young/' + str(j) + '.jpg', Yo[j])
                cv2.imwrite(str(epoch) + '_' + str(i) + '/young2old/' + str(j) + '_actual.jpg', Xy[j])
                cv2.imwrite(str(epoch) + '_' + str(i) + '/old2young/' + str(j) + '_actual.jpg', Xo[j])
                            
            model_D.save_weights('/content/drive/My Drive/UTK_inthewild/model_D.h5')
            model_E.save_weights('/content/drive/My Drive/UTK_inthewild/model_E.h5')
            model_G.save_weights('/content/drive/My Drive/UTK_inthewild/model_G.h5')
            model_H.save_weights('/content/drive/My Drive/UTK_inthewild/model_H.h5')
            gc.collect()

Epoch  0
1341/1618 [=======================>......] - ETA: 13:10 - loss: 4.5566 - generator loss: 4.1044 - discriminator loss: 0.4522

KeyboardInterrupt: ignored

In [0]:
model_D.save_weights('/content/drive/My Drive/UTK_inthewild/model_D_age_prev.h5')
model_E.save_weights('/content/drive/My Drive/UTK_inthewild/model_E_age_prev.h5')
model_G.save_weights('/content/drive/My Drive/UTK_inthewild/model_G_age_prev.h5')
model_H.save_weights('/content/drive/My Drive/UTK_inthewild/model_H_age_prev.h5')

In [0]:
def discriminator_loss(real, generated):
    real_loss = disc_loss_fn(tf.ones_like(real), real)
    generated_loss = disc_loss_fn(tf.zeros_like(generated), generated)
    total_disc_loss = real_loss + generated_loss
    return total_disc_loss

def cycled_loss(real, cycled):
    return cycled_loss_fn(real, cycled)

def identity_loss(real, same):
    return identity_loss_fn(real, same)

def generator_loss(generated):
  return gen_loss_fn(tf.ones_like(generated), generated)

@tf.function
def train(genAB, genBA, discA, discB, X1, X2, lambda1=10.0):
    real_A = X1
    real_B = X2
    with tf.GradientTape(persistent=True) as tape:
        fake_B = genAB(real_A)                  # Young -> old
        fake_A = genBA(real_B)                  # Old -> young
        
        cycled_A = genBA(fake_B)            # Young -> old -> young
        cycled_B = genAB(fake_A)            # old -> young -> old
        
        same_B = genAB(real_B)                 # Old -> Old
        same_A = genBA(real_A)                 # Young -> Young
        
        discA_fake_out = discA(fake_A)        # D predicts young, gen2, ie H, outputs young
        discB_fake_out = discB(fake_B)

        g_loss = mae_criterion(discA_fake_out, tf.ones_like(discA_fake_out)) \
                + mae_criterion(discB_fake_out, tf.ones_like(discB_fake_out)) \
                + lambda1 * abs_criterion(real_A, cycled_A) \
                + lambda1 * abs_criterion(real_B, cycled_B)  # \
                # + identity_loss(cycled_A, same_A) \
                # + identity_loss(cycled_B, same_B)
        
        # cycle_loss = cycled_loss(X1, cycled_x) + cycled_loss(X2, cycled_y)
        
        discA_real_out = discA(real_A)
        discB_real_out = discB(real_B)

        discA_loss_real = mae_criterion(discA_real_out, tf.ones_like(discA_real_out))
        discB_loss_real = mae_criterion(discB_real_out, tf.ones_like(discB_real_out))

        discA_loss_fake = mae_criterion(discA_fake_out, tf.zeros_like(discA_fake_out))
        discB_loss_fake = mae_criterion(discB_fake_out, tf.zeros_like(discB_fake_out))

        discA_loss = (discA_loss_real + discA_loss_fake) / 2.0
        discB_loss = (discB_loss_real + discB_loss_fake) / 2.0

        d_loss = discA_loss + discB_loss
                
        net_loss = g_loss + d_loss
           
    G_vars = genAB.trainable_variables
    gradients = tape.gradient(g_loss, G_vars)
    g1_optimizer.apply_gradients(zip(gradients, G_vars))
    
    H_vars = genBA.trainable_variables
    gradients = tape.gradient(g_loss, H_vars)
    g2_optimizer.apply_gradients(zip(gradients, H_vars))
    
    D_vars = discA.trainable_variables
    gradients = tape.gradient(d_loss, D_vars)
    d1_optimizer.apply_gradients(zip(gradients, D_vars))
    
    E_vars = discB.trainable_variables
    gradients = tape.gradient(d_loss, E_vars)
    d2_optimizer.apply_gradients(zip(gradients, E_vars))
    
    del tape
    gc.collect()
    return net_loss, g_loss, d_loss

In [0]:
g1_optimizer = tf.keras.optimizers.Nadam(learning_rate=5e-5, beta_1=0.5)
g2_optimizer = tf.keras.optimizers.Nadam(learning_rate=5e-5, beta_1=0.5)
d1_optimizer = tf.keras.optimizers.Nadam(learning_rate=5e-5, beta_1=0.5)
d2_optimizer = tf.keras.optimizers.Nadam(learning_rate=5e-5, beta_1=0.5)

gc.collect()
for epoch in range(10):
    
    print("Epoch ", epoch)
    batch_size=8
    steps = int((len(image_dataset['young']) + len(image_dataset['old']))/batch_size)
    progressbar = Progbar(steps)
    for i in range(steps) :
        # X1, X2 = create_young_old_batch(batch_size=32)
        # loss = train_discriminators(model_G, model_H, model_D, model_E, X1, X2)
        # gc.collect()
        X1, X2 = create_young_old_batch(batch_size=batch_size)
        loss, gl, dl = train(model_G, model_H, model_D, model_E, X1, X2)
        progressbar.update(i, [
            ('loss', loss.numpy()),
            ('generator loss', gl.numpy()),
            ('discriminator loss', dl.numpy())
        ])
        
        if i%100 == 0:
            os.mkdir(str(epoch) + '_' + str(i))
            os.mkdir(str(epoch) + '_' + str(i) + '/young2old')
            os.mkdir(str(epoch) + '_' + str(i) + '/old2young')
        
            Xy, Xo = create_young_old_test_batch(batch_size=8)
            Yy = np.around((1 + model_G.predict(Xy)) * 127.5)
            Yo = np.around((1 + model_H.predict(Xo)) * 127.5)
            Xy = np.around((1 + Xy) * 127.5)
            Xo = np.around((1 + Xo) * 127.5)
        
            for j in range(len(Yy)):
                cv2.imwrite(str(epoch) + '_' + str(i) + '/young2old/' + str(j) + '.jpg', Yy[j])
                cv2.imwrite(str(epoch) + '_' + str(i) + '/old2young/' + str(j) + '.jpg', Yo[j])
                cv2.imwrite(str(epoch) + '_' + str(i) + '/young2old/' + str(j) + '_actual.jpg', Xy[j])
                cv2.imwrite(str(epoch) + '_' + str(i) + '/old2young/' + str(j) + '_actual.jpg', Xo[j])
                            
            model_D.save_weights('/content/drive/My Drive/UTK_inthewild/model_D_age.h5')
            model_E.save_weights('/content/drive/My Drive/UTK_inthewild/model_E_age.h5')
            model_G.save_weights('/content/drive/My Drive/UTK_inthewild/model_G_age.h5')
            model_H.save_weights('/content/drive/My Drive/UTK_inthewild/model_H_age.h5')
            gc.collect()

Epoch  0


W0811 17:15:03.478314 139857060358016 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


1475/1618 [==========================>...] - ETA: 5:16 - loss: 3.0554 - generator loss: 2.6923 - discriminator loss: 0.3631

In [0]:
g1_optimizer = tf.keras.optimizers.Nadam(learning_rate=1e-5, beta_1=0.5)
g2_optimizer = tf.keras.optimizers.Nadam(learning_rate=1e-5, beta_1=0.5)
d1_optimizer = tf.keras.optimizers.Nadam(learning_rate=1e-5, beta_1=0.5)
d2_optimizer = tf.keras.optimizers.Nadam(learning_rate=1e-5, beta_1=0.5)

gc.collect()
for epoch in range(10):
    
    print("Epoch ", epoch)
    batch_size=8
    steps = int((len(image_dataset['young']) + len(image_dataset['old']))/batch_size)
    progressbar = Progbar(steps)
    for i in range(steps) :
        # X1, X2 = create_young_old_batch(batch_size=32)
        # loss = train_discriminators(model_G, model_H, model_D, model_E, X1, X2)
        # gc.collect()
        X1, X2 = create_young_old_batch(batch_size=batch_size)
        loss, gl, dl = train(model_G, model_H, model_D, model_E, X1, X2)
        progressbar.update(i, [
            ('loss', loss.numpy()),
            ('generator loss', gl.numpy()),
            ('discriminator loss', dl.numpy())
        ])
        
        if i%100 == 0:
            os.mkdir(str(epoch) + '_' + str(i))
            os.mkdir(str(epoch) + '_' + str(i) + '/young2old')
            os.mkdir(str(epoch) + '_' + str(i) + '/old2young')
        
            Xy, Xo = create_young_old_test_batch(batch_size=8)
            Yy = np.around((1 + model_G.predict(Xy)) * 127.5)
            Yo = np.around((1 + model_H.predict(Xo)) * 127.5)
            Xy = np.around((1 + Xy) * 127.5)
            Xo = np.around((1 + Xo) * 127.5)
        
            for j in range(len(Yy)):
                cv2.imwrite(str(epoch) + '_' + str(i) + '/young2old/' + str(j) + '.jpg', Yy[j])
                cv2.imwrite(str(epoch) + '_' + str(i) + '/old2young/' + str(j) + '.jpg', Yo[j])
                cv2.imwrite(str(epoch) + '_' + str(i) + '/young2old/' + str(j) + '_actual.jpg', Xy[j])
                cv2.imwrite(str(epoch) + '_' + str(i) + '/old2young/' + str(j) + '_actual.jpg', Xo[j])
                            
            model_D.save_weights('/content/drive/My Drive/UTK_inthewild/model_D_age.h5')
            model_E.save_weights('/content/drive/My Drive/UTK_inthewild/model_E_age.h5')
            model_G.save_weights('/content/drive/My Drive/UTK_inthewild/model_G_age.h5')
            model_H.save_weights('/content/drive/My Drive/UTK_inthewild/model_H_age.h5')
            gc.collect()

Epoch  0


W0812 10:11:16.585262 139718129379200 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


 301/1618 [====>.........................] - ETA: 49:46 - loss: 2.7185 - generator loss: 2.3835 - discriminator loss: 0.3351

KeyboardInterrupt: ignored

In [0]:
model_D.save_weights('/content/drive/My Drive/UTK_inthewild/model_D_age_bkp.h5')
model_E.save_weights('/content/drive/My Drive/UTK_inthewild/model_E_age_bkp.h5')
model_G.save_weights('/content/drive/My Drive/UTK_inthewild/model_G_age_bkp.h5')
model_H.save_weights('/content/drive/My Drive/UTK_inthewild/model_H_age_bkp.h5')

In [0]:
# Evaluation cell

Xy, Xo = create_young_old_test_batch(batch_size=8)
Yy = np.around((1 + model_G.predict(Xy)) * 127.5)
Yo = np.around((1 + model_H.predict(Xo)) * 127.5)
Xy = np.around((1 + Xy) * 127.5)
Xo = np.around((1 + Xo) * 127.5)

for i in range(len(Xy)):
    cv2.imwrite(str(i)+"_young.jpg", Yo[i])
    cv2.imwrite(str(i)+"_young_orig.jpg", Xo[i])
    cv2.imwrite(str(i)+"_old.jpg", Yy[i])
    cv2.imwrite(str(i)+"_old_orig.jpg", Xy[i])